In [3]:
import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

In [4]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [5]:
message= "is your api the best or will i get better one's off the market??"

messages = [{"role":"user", "content": message}]

messages

[{'role': 'user',
  'content': "is your api the best or will i get better one's off the market??"}]

In [6]:
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
response.choices[0].message.content

'Short answer: there isn’t a single “best” API. OpenAI’s API is one of the top options for general-purpose language tasks, but other providers (and open‑source models you can self-host) can be better for specific needs like cost, privacy, latency, on‑prem deployment, or safety controls. The right choice depends on your use case.\n\nHow to decide what’s best for you\n- Use case and requirements\n  - General chat, code, reasoning, creative writing? OpenAI is a solid default.\n  - Need strict data privacy, on‑prem hosting, or no training data usage? look at self-hosted/open‑source options or providers with strong data controls.\n  - Very low latency or high-throughput needs? some providers or regional deployments may win.\n  - Need specialized tools (retrieval augmentation, plugins, fine-tuning, or domain-specific models)? compare on those capabilities.\n- Cost and pricing model\n  - Per‑token costs, versus monthly fees, rate limits, and any enterprise discounts.\n- Model quality and safe

In [9]:
sd = fetch_website_contents("https://theyardgym.com/")
print(sd)

The Yard Gym | Group Strength and Conditioning Gym

NEWS
TIMETABLE
BEHIND THE BRAND
APPAREL
My Cart
Own A Yard Gym
Start a trial
THE TRAINING
THE SPACE
FIND A YARD
FRANCHISE
VIP EXPERIENCE
Own A Yard Gym
Start a trial
THE TRAINING
THE SPACE
FIND A YARD
FRANCHISE
VIP EXPERIENCE
NEWS
TIMETABLE
BEHIND THE BRAND
APPAREL
My Cart
The Yard Gym | Group Strength and Conditioning Gym
Your browser does not support the video tag.
Explore
BOUTIQUE
STRENGTH
&
CONDITIONING SPACE,
CATERING TO ALL AGE
GROUPS & FITNESS LEVELS,
CULTIVATED, MODERN,
LIFESTYLE-ORIENTED.
THE
TRAINING
RIG
Prioritizing the development of power, strength and muscular endurance through functional and hypertrophy training.
TURF
The combination of conditioning and functional movements designed to improve cardiovascular endurance, boost overall physical fitness and enhance athletic performance.
++
Mat Pilates, Run Club, and Pay Day Game Day are designed to complement our core RIG and TURF programs by targeting key areas like core s

In [10]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [11]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

In [12]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [14]:
# Try this out, and then try for a few more websites

messages_for(sd)

[{'role': 'system',
  'content': '\nYou are a snarky assistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\n\nThe Yard Gym | Group Strength and Conditioning Gym\n\nNEWS\nTIMETABLE\nBEHIND THE BRAND\nAPPAREL\nMy Cart\nOwn A Yard Gym\nStart a trial\nTHE TRAINING\nTHE SPACE\nFIND A YARD\nFRANCHISE\nVIP EXPERIENCE\nOwn A Yard Gym\nStart a trial\nTHE TRAINING\nTHE SPACE\nFIND A YARD\nFRANCHISE\nVIP EXPERIENCE\nNEWS\nTIMETABLE\nBEHIND THE BRAND\nAPPAREL\nMy Cart\nThe Yard Gym | Group Strength and Conditioning Gym\nYour browser does not support the video tag.\nExplore\nBOUTIQUE\nSTRENGTH\n&\nCONDITIONING\xa0SPACE,\nCATERING TO

In [15]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [17]:
summarize("https://theyardgym.com/")

'# The Yard Gym: Where Strength Meets Swagger\n\nWelcome to The Yard Gym, a boutique strength and conditioning HQ designed for everyone from rookie fitness fanatics to seasoned sweat monks. They’re all about power, muscular endurance, and cardio that doesn’t scream “torture.”\n\n### What’s On Offer?\n- **RIG:** Bulk up and get strong, focusing on power and hypertrophy.\n- **TURF:** Cardio meets functional moves for a fitness boost that won’t leave you gasping for air (well, maybe a little).\n- **Extras:** Mat Pilates, Run Club, and bizarrely named Pay Day Game Day to target core and agility like a boss.\n\n### Franchise Frenzy\nThe founder, Dan Bova, wants you to join the cult—er, franchise—and become a "weapon operator" to spread the gospel of health and fitness. Because apparently, owning a Yard Gym is both profitable *and* life-enhancing.\n\n### Other Highlights\n- You can start a trial—because who doesn’t love a free trial of getting yelled at?\n- There’s an app (because everything

In [18]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [19]:
display_summary("https://theyardgym.com/")

# The Yard Gym: Where Group Strength Meets Snarky Endurance

Welcome to The Yard Gym, a boutique fitness playground for all ages and sweat levels, specializing in group strength and conditioning. They've got fancy zones like the RIG for bulking up with functional and hypertrophy training, and the TURF for cardio beasts who want to run hard and move smart. Oh, and sprinkle in some Pilates, Run Club, and Pay Day Game Day for that extra core and agility zest.

**News & Announcements?**  
Nothing earth-shaking here—no flashing news banners or urgent updates, just the usual calls to "Start a trial," "Own a Yard Franchise," and bask in their community-driven, "We succeed when you succeed" mantra from founder Dan Bova. So basically, if you want to sweat, bond, and maybe buy some apparel or franchises, this is your turf.

In short: It’s a modern, lifestyle-oriented gym with a side of franchise ambitions and group camaraderie, perfect for those who love doing it all *together.* Sweat, buy, repeat.